# Importing the Required Libraries

In [2]:
! pip install pandas
print("Successfully Installed Pandas")

   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
    --------------------------------------- 0.3/11.0 MB ? eta -:--:--
    --------------------------------------- 0.3/11.0 MB ? eta -:--:--
   -- ------------------------------------- 0.8/11.0 MB 840.2 kB/s eta 0:00:13
   -- ------------------------------------- 0.8/11.0 MB 840.2 kB/s eta 0:00:13
   --- ------------------------------------ 1.0/11.0 MB 806.0 kB/s eta 0:00:13
   --- ------------------------------------ 1.0/11.0 MB 806.0 kB/s eta 0:00:13
   ---- ----------------------------------- 1.3/11.0 MB 784.8 kB/s eta 0:00:13
   ----- ---------------------------------- 1.6/11.0 MB 773.1 kB/s eta 0:00:13
   ----- ---------------------------------- 1.6/11.0 MB 773.1 kB/s eta 0:00:13
   ------ --------------------------------- 1.8/11.0 MB 767.7 kB/s eta 0:00:12
   ------ --------------------------------- 1.8/11.0 MB 767.7 kB/s eta 0:00:12
   ------


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
!pip install PyPDF2
print("Successfully Installed PyPDF2")

Successfully Installed PyPDF2



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import os
import glob
import pandas as pd

# For Pdf reading
from PyPDF2 import PdfReader

# For cleaning
import re

# Set warnings off

import warnings
warnings.filterwarnings("ignore")

print("All successfully imported")



All successfully imported


In [2]:
# Set paths for our project data
raw_data_path = "D:\Downloads\BangladeshLaw"
processed_data_path = "D:/Law-RAG/data/processed"

# Create processed folder if it doesn't exist
os.makedirs(processed_data_path, exist_ok=True)

# List all PDF/TXT files in raw data
file_list = glob.glob(os.path.join(raw_data_path, "*.pdf")) + \
            glob.glob(os.path.join(raw_data_path, "*.txt"))

print(f"Found {len(file_list)} files to process")
file_list[:5]  # show first 5 files


Found 46 files to process


["D:\\Downloads\\BangladeshLaw\\1872_ marriedwomen'sproperty_6.pdf",
 'D:\\Downloads\\BangladeshLaw\\1872_christianmarriage_3.pdf',
 'D:\\Downloads\\BangladeshLaw\\1872_contract_2.pdf',
 'D:\\Downloads\\BangladeshLaw\\1872_foreign_recruting_7.pdf',
 'D:\\Downloads\\BangladeshLaw\\1872_majority_9.pdf']

In [11]:
! pip install pdf2image pytesseract


  Using cached pdf2image-1.17.0-py3-none-any.whl.metadata (6.2 kB)
  Using cached pytesseract-0.3.13-py3-none-any.whl.metadata (11 kB)
Using cached pdf2image-1.17.0-py3-none-any.whl (11 kB)
Using cached pytesseract-0.3.13-py3-none-any.whl (14 kB)

   -------------------- ------------------- 1/2 [pdf2image]
   ---------------------------------------- 2/2 [pdf2image]




[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


## Extracting Text from Scanned PDFs Using OCR

This code converts each page of a scanned PDF into an image using pdf2image and Poppler.
Then it applies Tesseract OCR to recognize and extract text from those images.
Finally, it combines the extracted text into a single string for further use or analysis.

In [3]:
from pdf2image import convert_from_path
import pytesseract

pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

def read_pdf_ocr(file_path):
    """
    Reads a scanned PDF using OCR and returns combined text
    """
    text = ""
    pages = convert_from_path(file_path, dpi=300, poppler_path=r"D:\Downloads\Release-25.07.0-0\poppler-25.07.0\Library\bin")
    for page in pages:
        text += pytesseract.image_to_string(page, lang='eng') + "\n"
    return text

# Test
sample_pdf = [f for f in file_list if f.endswith(".pdf")][0]
sample_text = read_pdf_ocr(sample_pdf)
print(sample_text[:500])


30/08/2025 The Married Women's Property Act, 1874

The Married Women's Property Act, 1874
(ACT NO. Ill OF 1874 )

2#An Act to explain and amend the law relating to certain married women, and for other purposes.

Preamble

WHEREAS it is expedient to make such provision as hereinafter appears for the enjoyment of wages and earnings by women
married before the first day of January, 1866, and for insurances on lives by persons married before or after that day:

And whereas by the [Succession Act, 19


In [20]:
! pip uninstall fitz -y


Found existing installation: fitz 0.0.1.dev2
Uninstalling fitz-0.0.1.dev2:
  Successfully uninstalled fitz-0.0.1.dev2


In [21]:
! pip install PyMuPDF


  Using cached pymupdf-1.26.4-cp39-abi3-win_amd64.whl.metadata (3.4 kB)
Using cached pymupdf-1.26.4-cp39-abi3-win_amd64.whl (18.7 MB)



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
# 📌 Cell: OCR PDFs and save as TXT (Robust Version with Absolute Paths)
import os
import fitz  # pip install PyMuPDF
from PIL import Image
import pytesseract

# ✅ Absolute folder paths
raw_folder = r"D:\Downloads\BangladeshLaw"
processed_folder = r"D:\Law-RAG\data\processed"

# ✅ Create folders if they don't exist
os.makedirs(raw_folder, exist_ok=True)
os.makedirs(processed_folder, exist_ok=True)

# ✅ Get all PDF files in raw folder
pdf_files = [os.path.join(raw_folder, f) for f in os.listdir(raw_folder) if f.endswith(".pdf")]
print(f"Found {len(pdf_files)} PDF files to process")

# ✅ Loop through PDFs and save OCR text
for pdf_file in pdf_files:
    try:
        doc = fitz.open(pdf_file)
        text = ""
        for page in doc:
            # Convert page to image
            pix = page.get_pixmap(dpi=300)
            img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
            # OCR using pytesseract
            page_text = pytesseract.image_to_string(img, lang='eng')  # change 'eng' if needed
            text += page_text + "\n"

        # Save OCR text as TXT
        txt_file_name = os.path.join(processed_folder, os.path.basename(pdf_file).replace(".pdf", ".txt"))
        with open(txt_file_name, "w", encoding="utf-8") as f:
            f.write(text)

        print(f"✅ Saved OCR text for: {os.path.basename(pdf_file)}")

    except Exception as e:
        print(f"⚠ Skipping {os.path.basename(pdf_file)} due to error: {e}")


Found 46 PDF files to process
✅ Saved OCR text for: 1872_ marriedwomen'sproperty_6.pdf
✅ Saved OCR text for: 1872_christianmarriage_3.pdf
✅ Saved OCR text for: 1872_contract_2.pdf
✅ Saved OCR text for: 1872_foreign_recruting_7.pdf
✅ Saved OCR text for: 1872_majority_9.pdf
✅ Saved OCR text for: 1872_marriage_1.pdf
✅ Saved OCR text for: 1872_oaths_5.pdf
✅ Saved OCR text for: 1872_savingbanks_4.pdf
✅ Saved OCR text for: 1872_survey_8.pdf
✅ Saved OCR text for: 1875_law_reports_1.pdf
✅ Saved OCR text for: 1876_irrigation_1.pdf
✅ Saved OCR text for: 1877_specific_1.pdf
✅ Saved OCR text for: 1878_arms_1.pdf
✅ Saved OCR text for: 1878_tresuretrove_1.pdf
✅ Saved OCR text for: 1879_court_of_wards_1.pdf
✅ Saved OCR text for: 1879_touts_1.pdf
✅ Saved OCR text for: 1880_kazis_3.pdf
✅ Saved OCR text for: 1880_religious_societies_1.pdf
✅ Saved OCR text for: 1880_vaccination_2.pdf
✅ Saved OCR text for: 1881_chittagong_hill_tracts_police_regulation_1.pdf
✅ Saved OCR text for: 1881_municipal_taxation_2.

In [5]:
# 📌 Cell 5: Read processed TXT files and preprocess
import os
import re

processed_folder = r"D:\Law-RAG\data\processed"

# Get all TXT files
txt_files = [os.path.join(processed_folder, f) for f in os.listdir(processed_folder) if f.endswith(".txt")]
print(f"Found {len(txt_files)} TXT files to read")

# Function to clean text
def clean_text(text):
    # Remove multiple spaces, newlines
    text = re.sub(r'\s+', ' ', text)
    # Optional: remove non-ascii chars
    text = re.sub(r'[^\x00-\x7F]+', ' ', text)
    return text.strip()

# Read and preprocess all TXT files
all_texts = {}
for txt_file in txt_files:
    with open(txt_file, "r", encoding="utf-8") as f:
        text = f.read()
        clean = clean_text(text)
        all_texts[os.path.basename(txt_file)] = clean

print("✅ All TXT files read and cleaned!")
print("Sample text from first file:")
first_file = list(all_texts.keys())[0]
print(all_texts[first_file][:500])


Found 46 TXT files to read
✅ All TXT files read and cleaned!
Sample text from first file:
30/08/2025 The Married Women's Property Act, 1874 The Married Women's Property Act, 1874 (ACT NO. Ill OF 1874 ) 2#An Act to explain and amend the law relating to certain married women, and for other purposes. Preamble WHEREAS it is expedient to make such provision as hereinafter appears for the enjoyment of wages and earnings by women married before the first day of January, 1866, and for insurances on lives by persons married before or after that day: And whereas by the [Succession Act, 1925], 


In [6]:
# 📌 Cell 6: Chunking the cleaned text
def chunk_text(text, chunk_size=800, overlap=200):
    """
    Splits text into chunks with overlap for context.
    Example: If chunk_size=800 and overlap=200, 
    each chunk will have 800 chars but 200 chars overlap with previous.
    """
    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunk = text[start:end]
        chunks.append(chunk.strip())
        start += chunk_size - overlap  # move by chunk_size - overlap
    return chunks

# Apply chunking to all cleaned texts
chunked_texts = {}
total_chunks = 0
for filename, text in all_texts.items():
    chunks = chunk_text(text)
    chunked_texts[filename] = chunks
    total_chunks += len(chunks)

print(f"✅ Chunking complete! Created {total_chunks} chunks from {len(all_texts)} documents.")
print("Sample chunks from first file:")
first_file = list(chunked_texts.keys())[0]
for i, chunk in enumerate(chunked_texts[first_file][:3]):  # show first 3 chunks
    print(f"\n--- Chunk {i+1} ---\n{chunk[:400]}...")


✅ Chunking complete! Created 2260 chunks from 46 documents.
Sample chunks from first file:

--- Chunk 1 ---
30/08/2025 The Married Women's Property Act, 1874 The Married Women's Property Act, 1874 (ACT NO. Ill OF 1874 ) 2#An Act to explain and amend the law relating to certain married women, and for other purposes. Preamble WHEREAS it is expedient to make such provision as hereinafter appears for the enjoyment of wages and earnings by women married before the first day of January, 1866, and for insuranc...

--- Chunk 2 ---
e person whom he or she marries, nor become incapable of doing any act in respect of his or her own property, which he or she could have done, if unmarried: And whereas by force of the said Act all women to whose marriages it applies are absolute owners of all property vested in, or acquired by, them, and their husbands do not by their marriage acquire any interest in such property, but the said A...

--- Chunk 3 ---
ves: CHAPTER | PRELIMINARY Short title 1. This Act

In [27]:
! pip install sentence-transformers


  Using cached sentence_transformers-5.1.0-py3-none-any.whl.metadata (16 kB)
  Using cached transformers-4.56.1-py3-none-any.whl.metadata (42 kB)
  Using cached huggingface_hub-0.34.4-py3-none-any.whl.metadata (14 kB)
  Using cached tokenizers-0.22.0-cp39-abi3-win_amd64.whl.metadata (6.9 kB)
  Using cached safetensors-0.6.2-cp38-abi3-win_amd64.whl.metadata (4.1 kB)
Using cached sentence_transformers-5.1.0-py3-none-any.whl (483 kB)
Using cached transformers-4.56.1-py3-none-any.whl (11.6 MB)
Using cached huggingface_hub-0.34.4-py3-none-any.whl (561 kB)
Using cached tokenizers-0.22.0-cp39-abi3-win_amd64.whl (2.7 MB)
Using cached safetensors-0.6.2-cp38-abi3-win_amd64.whl (320 kB)

   ---------------------------------------- 0/7 [tqdm]
   ---------------------------------------- 0/7 [tqdm]
   ---------------------------------------- 0/7 [tqdm]
   ----- ---------------------------------- 1/7 [safetensors]
   ----------- ---------------------------- 2/7 [regex]
   ----------------- ----------


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
from sentence_transformers import SentenceTransformer

# Load small fast model
model = SentenceTransformer("all-MiniLM-L6-v2")

# Test sentences
sentences = ["Hello world", "Testing embeddings"]

# Generate embeddings
embeddings = model.encode(sentences)

print("Shape:", embeddings.shape)
print("First embedding vector (truncated):", embeddings[0][:10])


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


Shape: (2, 384)
First embedding vector (truncated): [-0.0344772   0.03102322  0.006735    0.02610897 -0.03936201 -0.16030249
  0.06692398 -0.00644147 -0.04745052  0.01475886]


In [8]:
from sentence_transformers import SentenceTransformer
import numpy as np
embed_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
all_embeddings = {}
for filename, chunks in chunked_texts.items():
    print(f"Embedding {filename} with {len(chunks)} chunks...")
    embeddings = embed_model.encode(chunks, batch_size=16, show_progress_bar=True)
    all_embeddings[filename] = embeddings
print("Embeddings completed")    
    
    
    




Embedding 1872_ marriedwomen'sproperty_6.txt with 16 chunks...


Batches: 100%|███████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.72it/s]


Embedding 1872_christianmarriage_3.txt with 95 chunks...


Batches: 100%|███████████████████████████████████████████████████████████████████████████| 6/6 [00:03<00:00,  1.91it/s]


Embedding 1872_contract_2.txt with 0 chunks...


...

Embedding 1896_the protection_of_muslim_pilgrims_1.txt with 14 chunks...


Batches: 100%|███████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.63it/s]


Embedding 1897_general_clause_1.txt with 84 chunks...


Batches: 100%|███████████████████████████████████████████████████████████████████████████| 6/6 [00:03<00:00,  1.69it/s]

Embeddings completed


In [10]:
# Save embeddings as numpy files for persistence
import os

os.makedirs("data/embeddings", exist_ok=True)

for filename, embeddings in all_embeddings.items():
    np.save(f"data/embeddings/{filename}_embeddings.npy", embeddings)


In [11]:
import os

# List all files inside embeddings folder
print(os.listdir("data/embeddings"))


["1872_ marriedwomen'sproperty_6.txt_embeddings.npy", '1872_christianmarriage_3.txt_embeddings.npy', '1872_contract_2.txt_embeddings.npy', '1872_foreign_recruting_7.txt_embeddings.npy', '1872_majority_9.txt_embeddings.npy', '1872_marriage_1.txt_embeddings.npy', '1872_oaths_5.txt_embeddings.npy', '1872_savingbanks_4.txt_embeddings.npy', '1872_survey_8.txt_embeddings.npy', '1875_law_reports_1.txt_embeddings.npy', '1876_irrigation_1.txt_embeddings.npy', '1877_specific_1.txt_embeddings.npy', '1878_arms_1.txt_embeddings.npy', '1878_tresuretrove_1.txt_embeddings.npy', '1879_court_of_wards_1.txt_embeddings.npy', '1879_touts_1.txt_embeddings.npy', '1880_kazis_3.txt_embeddings.npy', '1880_religious_societies_1.txt_embeddings.npy', '1880_vaccination_2.txt_embeddings.npy', '1881_chittagong_hill_tracts_police_regulation_1.txt_embeddings.npy', '1881_municipal_taxation_2.txt_embeddings.npy', '1881_negotiable_instruments_4.txt_embeddings.npy', '1881_obstructions_in_fairway_3.txt_embeddings.npy', '188

In [13]:
! pip install faiss-cpu
print("Successfully installed faiss")

   ---------------------------------------- 0.0/18.2 MB ? eta -:--:--
    --------------------------------------- 0.3/18.2 MB ? eta -:--:--
   --------- ------------------------------ 4.2/18.2 MB 16.3 MB/s eta 0:00:01
   --------------------------- ------------ 12.6/18.2 MB 28.1 MB/s eta 0:00:01
   ---------------------------------------- 18.2/18.2 MB 30.5 MB/s  0:00:00


In [16]:
import numpy as np
import os

embedding_folder = "data/embeddings"
embedding_files = [f for f in os.listdir(embedding_folder) if f.endswith(".npy")]

dims = {}
for file in embedding_files:
    emb = np.load(os.path.join(embedding_folder, file))
    if len(emb.shape) == 1:  # single vector
        emb = emb.reshape(1, -1)
    dims[file] = emb.shape

print("📊 Embedding dimensions per file:")
for file, shape in dims.items():
    print(f"{file}: {shape}")

unique_dims = set(shape[1] for shape in dims.values())
print("\n🔎 Unique embedding dimensions found:", unique_dims)


📊 Embedding dimensions per file:
1872_ marriedwomen'sproperty_6.txt_embeddings.npy: (16, 384)
1872_christianmarriage_3.txt_embeddings.npy: (95, 384)
1872_contract_2.txt_embeddings.npy: (1, 0)
1872_foreign_recruting_7.txt_embeddings.npy: (6, 384)
1872_majority_9.txt_embeddings.npy: (6, 384)
1872_marriage_1.txt_embeddings.npy: (25, 384)
1872_oaths_5.txt_embeddings.npy: (14, 384)
1872_savingbanks_4.txt_embeddings.npy: (17, 384)
1872_survey_8.txt_embeddings.npy: (99, 384)
1875_law_reports_1.txt_embeddings.npy: (3, 384)
1876_irrigation_1.txt_embeddings.npy: (115, 384)
1877_specific_1.txt_embeddings.npy: (136, 384)
1878_arms_1.txt_embeddings.npy: (55, 384)
1878_tresuretrove_1.txt_embeddings.npy: (23, 384)
1879_court_of_wards_1.txt_embeddings.npy: (113, 384)
1879_touts_1.txt_embeddings.npy: (15, 384)
1880_kazis_3.txt_embeddings.npy: (8, 384)
1880_religious_societies_1.txt_embeddings.npy: (12, 384)
1880_vaccination_2.txt_embeddings.npy: (46, 384)
1881_chittagong_hill_tracts_police_regulation_1

In [17]:
import numpy as np
import os

embedding_folder = "data/embeddings"
embedding_files = [f for f in os.listdir(embedding_folder) if f.endswith(".npy")]

clean_embeddings = {}

for file in embedding_files:
    emb = np.load(os.path.join(embedding_folder, file))
    
    # If single vector, reshape
    if len(emb.shape) == 1:
        emb = emb.reshape(1, -1)
    
    # Filter out empty vectors
    if emb.shape[1] == 0:
        print(f"⚠️ Empty embedding found in {file}, skipping...")
        continue
    
    clean_embeddings[file] = emb

print(f"✅ Ready embeddings for FAISS. {len(clean_embeddings)} files will be indexed.")


⚠️ Empty embedding found in 1872_contract_2.txt_embeddings.npy, skipping...
✅ Ready embeddings for FAISS. 45 files will be indexed.


In [18]:
import faiss

# Initialize FAISS index
d = list(clean_embeddings.values())[0].shape[1]  # embedding dimension
index = faiss.IndexFlatL2(d)  # L2 similarity

# Metadata to know which chunk belongs to which file
metadata = []

for file, emb in clean_embeddings.items():
    index.add(emb)
    num_chunks = emb.shape[0]
    metadata.extend([(file, i) for i in range(num_chunks)])

print(f"✅ FAISS index created with {index.ntotal} vectors.")
print(f"Sample metadata entries: {metadata[:5]}")


✅ FAISS index created with 2260 vectors.
Sample metadata entries: [("1872_ marriedwomen'sproperty_6.txt_embeddings.npy", 0), ("1872_ marriedwomen'sproperty_6.txt_embeddings.npy", 1), ("1872_ marriedwomen'sproperty_6.txt_embeddings.npy", 2), ("1872_ marriedwomen'sproperty_6.txt_embeddings.npy", 3), ("1872_ marriedwomen'sproperty_6.txt_embeddings.npy", 4)]


In [22]:
import faiss
import numpy as np

# --- Step 1: Build FAISS index ---
# Get embedding dimension from first non-empty embedding
for emb in all_embeddings.values():
    if emb.size > 0:
        dimension = emb.shape[1] if len(emb.shape) > 1 else len(emb)
        break

index = faiss.IndexFlatL2(dimension)
metadata = []

for file, emb in all_embeddings.items():
    emb = np.array(emb, dtype='float32')

    # Skip empty embeddings
    if emb.size == 0:
        print(f"⚠️ Skipping empty embeddings: {file}")
        continue

    # Ensure 2D
    if len(emb.shape) == 1:
        emb = emb.reshape(1, -1)

    # Skip embeddings with wrong dimension
    if emb.shape[1] != dimension:
        print(f"⚠️ Skipping {file}, dimension mismatch: {emb.shape[1]} vs {dimension}")
        continue

    index.add(emb)

    # Add metadata
    base_file = file.replace("_embeddings.npy", "")
    for i in range(len(emb)):
        metadata.append((base_file, i))

print(f"✅ FAISS index built with {index.ntotal} vectors.")


⚠️ Skipping empty embeddings: 1872_contract_2.txt
✅ FAISS index built with 2260 vectors.


In [27]:
def search(query, top_k=3):
    """
    Search the FAISS index for the top_k most similar chunks to the query.
    """
    # Get embedding for query
    query_emb = embed_model.encode([query], convert_to_numpy=True, normalize_embeddings=True)
    query_emb = np.array(query_emb, dtype='float32')

    # Make sure it's 2D
    if len(query_emb.shape) == 1:
        query_emb = query_emb.reshape(1, -1)

    # Search FAISS
    distances, indices = index.search(query_emb, top_k)

    results = []
    for dist, idx in zip(distances[0], indices[0]):
        file, chunk_id = metadata[idx]
        chunk_text = chunked_texts[file][chunk_id]
        results.append({
            "file": file,
            "chunk_id": chunk_id,
            "distance": dist,
            "text": chunk_text[:500]  # show first 500 chars
        })
    return results


In [28]:
query_text = "marriage property rights"
results = search(query_text, top_k=3)

for i, res in enumerate(results):
    print(f"\n--- Result {i+1} ---")
    print(f"File: {res['file']}, Chunk: {res['chunk_id']}, Distance: {res['distance']}")
    print(res['text'])



--- Result 1 ---
File: 1872_ marriedwomen'sproperty_6.txt, Chunk: 10, Distance: 0.7936227917671204
d. 8. If a married woman (whether married before or after the first day of January, 1866) possesses separate property, and if any person enters into a contract with her with reference to such property, or on the faith that her obligation arising out of such contract will be satisfied out of her separate property, such person shall be entitled to sue her, and, to the extent of her separate property, to recover against her whatever he might have recovered in such suit had she been unmarried at the

--- Result 2 ---
File: 1882_property_1.txt, Chunk: 56, Distance: 0.81392502784729
he income of the property is insufficient for A's maintenance, and that the sale of the field is necessary, and, acting in good faith, buys the field from A. As between B on the one part and A and the collateral heirs on the other part, a necessity for the sale shall be deemed to have existed. 39. Where a third per

In [29]:
def batch_search(queries, top_k=3):
    """
    Search multiple queries at once and return top_k results for each.
    """
    results_all = {}
    for query in queries:
        results = search(query, top_k=top_k)
        results_all[query] = results
    return results_all

# Example usage:
queries = [
    "marriage property rights",
    "contract obligations",
    "religious societies regulations"
]

results_dict = batch_search(queries, top_k=3)

# Print results
for query, results in results_dict.items():
    print(f"\n🔹 Query: {query}")
    for i, res in enumerate(results):
        print(f"\n--- Result {i+1} ---")
        print(f"File: {res['file']}, Chunk: {res['chunk_id']}, Distance: {res['distance']}")
        print(res['text'])



🔹 Query: marriage property rights

--- Result 1 ---
File: 1872_ marriedwomen'sproperty_6.txt, Chunk: 10, Distance: 0.7936227917671204
d. 8. If a married woman (whether married before or after the first day of January, 1866) possesses separate property, and if any person enters into a contract with her with reference to such property, or on the faith that her obligation arising out of such contract will be satisfied out of her separate property, such person shall be entitled to sue her, and, to the extent of her separate property, to recover against her whatever he might have recovered in such suit had she been unmarried at the

--- Result 2 ---
File: 1882_property_1.txt, Chunk: 56, Distance: 0.81392502784729
he income of the property is insufficient for A's maintenance, and that the sale of the field is necessary, and, acting in good faith, buys the field from A. As between B on the one part and A and the collateral heirs on the other part, a necessity for the sale shall be deemed to 

In [30]:
def interactive_search(top_k=3):
    """
    Interactive search: type a query and get top_k results.
    Type 'exit' to quit.
    """
    print("🔎 Welcome to the interactive search! Type 'exit' to quit.\n")
    
    while True:
        query = input("Enter your search query: ")
        if query.lower() == "exit":
            print("Exiting interactive search.")
            break
        
        results = search(query, top_k=top_k)
        if not results:
            print("No results found.\n")
            continue
        
        print(f"\nTop {top_k} results for: '{query}'")
        for i, res in enumerate(results):
            print(f"\n--- Result {i+1} ---")
            print(f"File: {res['file']}, Chunk: {res['chunk_id']}, Distance: {res['distance']:.4f}")
            print(res['text'][:500])  # show first 500 chars
        print("\n" + "-"*50 + "\n")

# Run the interactive search
interactive_search(top_k=3)


🔎 Welcome to the interactive search! Type 'exit' to quit.



Enter your search query:  Marriage rules



Top 3 results for: 'Marriage rules'

--- Result 1 ---
File: 1872_marriage_1.txt, Chunk: 17, Distance: 0.8358
person married under this Act who, during the life time of his or her wife or husband, contracts any other marriage, shall be subject to the penalties provided in sections 494 and 495 of the Penal Code for the offence of marrying again during the lifetime of a husband or wife, whatever may be the religion which he or she professed at the time of such second marriage. 17. The Divorce Act shall apply to all marriages contracted under this Act, and any such marriage may be declared null or dissolve

--- Result 2 ---
File: 1872_marriage_1.txt, Chunk: 16, Distance: 0.9174
ook shall at all reasonable times be open for inspection, and shall be admissible as evidence of the truth of the statements therein contained. Certified extracts therefrom shall on application be given by the Registrar on the payment to him by the applicant of a fee to be fixed by the Government for each such extr

Enter your search query:  exit


Exiting interactive search.


In [31]:
def interactive_search_full(top_k=3):
    """
    Interactive search: type a query and get top_k results.
    Shows the full text of each matching chunk.
    Type 'exit' to quit.
    """
    print("🔎 Welcome to the interactive search! Type 'exit' to quit.\n")
    
    while True:
        query = input("Enter your search query: ")
        if query.lower() == "exit":
            print("Exiting interactive search.")
            break
        
        results = search(query, top_k=top_k)
        if not results:
            print("No results found.\n")
            continue
        
        print(f"\nTop {top_k} results for: '{query}'")
        for i, res in enumerate(results):
            print(f"\n--- Result {i+1} ---")
            print(f"File: {res['file']}, Chunk: {res['chunk_id']}, Distance: {res['distance']:.4f}")
            print(res['text'])  # show full chunk
        print("\n" + "-"*50 + "\n")

# Run the interactive search with full chunks
interactive_search_full(top_k=3)


🔎 Welcome to the interactive search! Type 'exit' to quit.



Enter your search query:  Labour law



Top 3 results for: 'Labour law'

--- Result 1 ---
File: 1890_railway_act_1.txt, Chunk: 85, Distance: 0.9799
s (Revision And Declaration) Act, 1973 (Act No. VIII of 1973).] 28/59 05/09/2025 Limitation of hours of work Grant of periodical rest bdlaws.minlaw.gov.bd/act-print-65.html The Railways Act, 1890 71C. (1) The railway servant, other than the railway servant whose employment is essentially intermittent, shall not be employed for more than sixty hours a week on the average in any month. (2) The railway servant whose employment is essentially intermittent shall not be employed for more than eighty-f

--- Result 2 ---
File: 1890_railway_act_1.txt, Chunk: 86, Distance: 0.9808
servants from the provisions of sub-section (1) and sub-section (2) may be made- (a) when such temporary exemptions are necessary to avoid serious interference with the ordinary working of the railway, in cases of accident, actual or threatened, or when urgent work is required to be done to the railway or to roll

Enter your search query:  exit


Exiting interactive search.


In [36]:
! pip install nltk

   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.5 MB ? eta -:--:--
   ---------------------------------------- 1.5/1.5 MB 8.2 MB/s  0:00:00


In [37]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

def chunk_text_sentences(text, chunk_size=800, overlap=200):
    """
    Splits text into chunks of approx chunk_size chars, ending at sentence boundaries,
    with specified overlap for context.
    """
    sentences = sent_tokenize(text)
    chunks = []
    current_chunk = ""
    
    for sent in sentences:
        if len(current_chunk) + len(sent) + 1 <= chunk_size:
            current_chunk += " " + sent
        else:
            chunks.append(current_chunk.strip())
            # Start next chunk with overlap
            current_chunk = " ".join(sentences[max(0, sentences.index(sent)-overlap//chunk_size):sentences.index(sent)+1])
    
    if current_chunk:
        chunks.append(current_chunk.strip())
    
    return chunks


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


In [38]:
def display_search_results(results, chunked_texts, context_chunks=1):
    """
    Display search results with neighboring chunk context.
    
    results: list of dicts from search function
    chunked_texts: dict of all chunked texts
    context_chunks: how many previous and next chunks to show
    """
    for i, res in enumerate(results):
        file = res["file"]
        chunk_id = res["chunk_id"]
        distance = res["distance"]
        
        # Determine the chunk range for context
        start = max(0, chunk_id - context_chunks)
        end = min(len(chunked_texts[file]), chunk_id + context_chunks + 1)
        
        # Combine chunks into one string
        combined_text = " ".join(chunked_texts[file][start:end])
        
        print(f"\n--- Result {i+1} ---")
        print(f"File: {file}, Chunk: {chunk_id}, Distance: {distance:.4f}")
        print(combined_text[:1000] + "...")  # show first 1000 chars for readability

# Example usage
query_text = "marriage property rights"
results = search(query_text, top_k=3)
display_search_results(results, chunked_texts)



--- Result 1 ---
File: 1872_ marriedwomen'sproperty_6.txt, Chunk: 10, Distance: 0.7936
re the first day of April, 1923. CHAPTER IV LEGAL PROCEEDINGS BY AND AGAINST MARRIED WOMEN 7. A married woman may maintain a suit in her own name for the recovery of property of any description which by force of the said  [Succession Act, 1925], or of this Act, is her separate property; and she shall have, in her own name, the same remedies, both civil and criminal, against all persons, for the protection and security of such property, as if she were unmarried, and she shall be liable to such suits, processes and orders in respect of such property as she would be liable to if she were unmarried. 8. If a married woman (whether married before or after the first day of January, 1866) possesses separate property, and if any person enters into a contract with her with reference to such property d. 8. If a married woman (whether married before or after the first day of January, 1866) possesses separate pr

In [39]:
import re

def highlight_text(text, query):
    """
    Highlights all occurrences of query words in the text (case-insensitive).
    """
    words = query.split()
    for word in words:
        # Use regex to match whole words, case-insensitive
        text = re.sub(f"(?i)({re.escape(word)})", r"\033[93m\1\033[0m", text)
    return text

def display_search_results_highlight(results, chunked_texts, query, context_chunks=1):
    """
    Display search results with neighboring chunk context and highlighted query.
    """
    for i, res in enumerate(results):
        file = res["file"]
        chunk_id = res["chunk_id"]
        distance = res["distance"]
        
        # Determine the chunk range for context
        start = max(0, chunk_id - context_chunks)
        end = min(len(chunked_texts[file]), chunk_id + context_chunks + 1)
        
        # Combine chunks into one string
        combined_text = " ".join(chunked_texts[file][start:end])
        highlighted_text = highlight_text(combined_text, query)
        
        print(f"\n--- Result {i+1} ---")
        print(f"File: {file}, Chunk: {chunk_id}, Distance: {distance:.4f}")
        print(highlighted_text[:1000] + "...")  # show first 1000 chars

# Example usage
query_text = "marriage property rights"
results = search(query_text, top_k=3)
display_search_results_highlight(results, chunked_texts, query_text)



--- Result 1 ---
File: 1872_ marriedwomen'sproperty_6.txt, Chunk: 10, Distance: 0.7936
re the first day of April, 1923. CHAPTER IV LEGAL PROCEEDINGS BY AND AGAINST MARRIED WOMEN 7. A married woman may maintain a suit in her own name for the recovery of property of any description which by force of the said  [Succession Act, 1925], or of this Act, is her separate property; and she shall have, in her own name, the same remedies, both civil and criminal, against all persons, for the protection and security of such property, as if she were unmarried, and she shall be liable to such suits, processes and orders in respect of such property as she would be liable to if she were unmarried. 8. If a married woman (whether married before or after the first day of January, 1866) possesses separate property, and if any person enters into a contract with her with reference to such property d. 8. If a married woman (whether married before or after the first day of January, 1866) possesses separate pr

In [40]:
import faiss
import pickle
import os

os.makedirs("data/index", exist_ok=True)

# Save FAISS index
faiss.write_index(index, "data/index/faiss_index.bin")
print("✅ FAISS index saved!")


✅ FAISS index saved!


In [41]:
# Save metadata dictionary (filename + chunk_id for each vector)
with open("data/index/metadata.pkl", "wb") as f:
    pickle.dump(metadata, f)

print("✅ Metadata saved!")


✅ Metadata saved!


In [42]:
# Load FAISS index
index = faiss.read_index("data/index/faiss_index.bin")

# Load metadata
with open("data/index/metadata.pkl", "rb") as f:
    metadata = pickle.load(f)

print("✅ FAISS index and metadata loaded!")


✅ FAISS index and metadata loaded!


In [43]:
# 🔹 Function to merge consecutive chunks into larger passages
def merge_chunks(chunks, max_merge=2):
    """
    Merge consecutive chunks for better context.
    max_merge = how many consecutive chunks to merge.
    """
    merged = []
    i = 0
    while i < len(chunks):
        merged_chunk = chunks[i]
        for j in range(1, max_merge):
            if i + j < len(chunks):
                merged_chunk += " " + chunks[i + j]
        merged.append(merged_chunk.strip())
        i += max_merge  # skip merged chunks
    return merged

# 🔹 Apply merged chunks to all documents
merged_chunked_texts = {}
for filename, chunks in chunked_texts.items():
    merged_chunked_texts[filename] = merge_chunks(chunks, max_merge=2)

print("✅ Merging chunks done. Sample merged chunks from first file:")
first_file = list(merged_chunked_texts.keys())[0]
for i, chunk in enumerate(merged_chunked_texts[first_file][:2]):
    print(f"\n--- Merged Chunk {i+1} ---\n{chunk[:500]}...")


✅ Merging chunks done. Sample merged chunks from first file:

--- Merged Chunk 1 ---
30/08/2025 The Married Women's Property Act, 1874 The Married Women's Property Act, 1874 (ACT NO. Ill OF 1874 ) 2#An Act to explain and amend the law relating to certain married women, and for other purposes. Preamble WHEREAS it is expedient to make such provision as hereinafter appears for the enjoyment of wages and earnings by women married before the first day of January, 1866, and for insurances on lives by persons married before or after that day: And whereas by the [Succession Act, 1925], ...

--- Merged Chunk 2 ---
ves: CHAPTER | PRELIMINARY Short title 1. This Act may be called the Married Women's Property Act, 1874. Extent and 2. It extends to the whole of Bangladesh. application bdlaws.minlaw.gov.bd/act-print-30.html 1/6 30/08/2025 [Repealed] Married women's earnings to be their separate property bdlaws.minlaw.gov.bd/act-print-30.html The Married Women's Property Act, 1874 But nothing herein 

In [44]:
with open("data/index/chunked_texts.pkl", "wb") as f:
    pickle.dump(chunked_texts, f)


In [45]:
import pickle

# Load the file
with open("data/index/chunked_texts.pkl", "rb") as f:
    loaded_chunks = pickle.load(f)

# Check type and keys
print(type(loaded_chunks))           # Should be <class 'dict'>
print(list(loaded_chunks.keys())[:5])  # Show first 5 filenames

# Inspect one file's chunks
first_file = list(loaded_chunks.keys())[0]
print(f"\nNumber of chunks in {first_file}: {len(loaded_chunks[first_file])}")
print("First chunk preview:\n", loaded_chunks[first_file][0][:500])  # first 500 chars


<class 'dict'>
["1872_ marriedwomen'sproperty_6.txt", '1872_christianmarriage_3.txt', '1872_contract_2.txt', '1872_foreign_recruting_7.txt', '1872_majority_9.txt']

Number of chunks in 1872_ marriedwomen'sproperty_6.txt: 16
First chunk preview:
 30/08/2025 The Married Women's Property Act, 1874 The Married Women's Property Act, 1874 (ACT NO. Ill OF 1874 ) 2#An Act to explain and amend the law relating to certain married women, and for other purposes. Preamble WHEREAS it is expedient to make such provision as hereinafter appears for the enjoyment of wages and earnings by women married before the first day of January, 1866, and for insurances on lives by persons married before or after that day: And whereas by the [Succession Act, 1925], 
